In [ ]:
import pandas as pd
import panel as pn

import plotly.express as px

pn.extension("plotly", sizing_mode="stretch_width")

## Styling Plotly for Panel

In this example we will show how to style Plotly plots with Panel supporting the `default` and the `dark` theme.

![PlotlyStyle.gif](../../assets/PlotlyStyle.gif)

## Get or set the theme

When we use the Fast templates the theme will be available from the query args

In [ ]:
def get_theme():
    return pn.state.session_args.get("theme", [b'default'])[0].decode()

In [ ]:
theme=get_theme()
theme

## Select a nice accent color

Below we create some functionality to *cycle through* a list of nice accent colors.

In [ ]:
nice_accent_colors = [
    ("#00A170", "white"), # Mint
    ("#DAA520", "white"), # Golden Rod
    ("#F08080", "white"), # Light Coral
    ("#4099da", "white"), # Summery Sky
    ("#2F4F4F", "white"), # Dark Slate Grey
    ("#A01346", "white"), # Fast
]

In [ ]:
def get_nice_accent_color():
    """Returns the 'next' nice accent color"""
    if not "color_index" in pn.state.cache:
        pn.state.cache["color_index"]=0
    elif pn.state.cache["color_index"]==len(nice_accent_colors)-1:
        pn.state.cache["color_index"]=0
    else:
        pn.state.cache["color_index"]+=1
    return nice_accent_colors[pn.state.cache["color_index"]]

In [ ]:
accent_color, color = get_nice_accent_color()
pn.pane.Markdown(f"# Color: {accent_color}", background=accent_color, height=70, margin=0, style={"color": color, "padding": "10px"})

## Plotly Express

Plotly Express enables you to specify a `template`. Lets create a widget for selecting the template

In [ ]:
plotly_template = pn.widgets.Select(options=["plotly", "plotly_dark"])

and set its default value

In [ ]:
if theme=="dark":
    plotly_template.value="plotly_dark"

In [ ]:
def get_plot(template=plotly_template.value, accent_color=accent_color):
    data = pd.DataFrame(
        [
            ("Monday", 7),
            ("Tuesday", 4),
            ("Wednesday", 9),
            ("Thursday", 4),
            ("Friday", 4),
            ("Saturday", 4),
            ("Sunay", 4),
        ],
        columns=["Day", "Orders"],
    )
    fig = px.line(
        data,
        x="Day",
        y="Orders",
        template=template,
        color_discrete_sequence=(accent_color,),
    )
    fig.update_traces(mode="lines+markers", marker=dict(size=10), line=dict(width=4))
    fig.layout.autosize = True
    return fig



Lets [bind](https://panel.holoviz.org/user_guide/APIs.html#reactive-functions) `get_plot` to the selected `template` and lay out the two in a `Column`.

In [ ]:
get_plot=pn.bind(get_plot, template=plotly_template)
plotly_component = pn.Column(plotly_template,pn.pane.panel(get_plot, config={"responsive": True}, sizing_mode="stretch_both", name="Plotly Express"), sizing_mode="stretch_both")
plotly_component

## Wrap it up in a nice template

Here we use the [`FastListTemplate`](https://panel.holoviz.org/reference/templates/FastListTemplate.html#templates-gallery-fastlisttemplate)

In [ ]:
template = pn.template.FastListTemplate(
    site="Panel",
    title="Styling Plotly",
    main=[plotly_component],
    accent_base_color=accent_color,
    header_background=accent_color,
    header_color=color
).servable();

You can serve the app via `panel serve PlotlyStyle.ipynb` and find it at `http://localhost:5006/PlotlyStyle`. You should add the `--autoreload` flag while developing for *hot reloading*.